# Momentum Trading Strategies

This notebook implements and analyzes various momentum trading strategies for high-frequency trading applications.

## Overview

Momentum strategies are based on the principle that assets which have performed well (or poorly) in the recent past will continue to perform well (or poorly) in the near future. This notebook explores:

- **Time Series Momentum**: Strategies based on an asset's own past returns
- **Cross-Sectional Momentum**: Relative performance strategies across multiple assets
- **Risk-Adjusted Momentum**: Incorporating volatility and risk metrics
- **Multi-Timeframe Momentum**: Combining signals from different time horizons

## Contents

1. Data Loading and Preprocessing
2. Momentum Indicator Calculation
3. Strategy Implementation
4. Backtesting and Performance Analysis
5. Risk Management and Position Sizing
6. Strategy Optimization and Parameter Tuning

## 1. Import Libraries and Setup

Import necessary libraries for data analysis, visualization, and strategy implementation.

In [ ]:
# Standard libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from datetime import datetime, timedelta

# Scientific computing
from scipy import stats
from scipy.optimize import minimize
import scipy.signal as signal

# Machine learning
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import sharpe_score

# Visualization
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots

# Trading and financial analysis
import yfinance as yf
import ta
import empyrical as ep

# Configuration
warnings.filterwarnings('ignore')
plt.style.use('seaborn-v0_8')
pd.set_option('display.max_columns', None)
pd.set_option('display.precision', 4)

# Random seed for reproducibility
np.random.seed(42)

## 2. Data Loading and Preprocessing

Load market data and prepare it for momentum strategy analysis.

In [ ]:
# Define parameters
SYMBOLS = ['AAPL', 'GOOGL', 'MSFT', 'AMZN', 'TSLA', 'META', 'NVDA', 'NFLX']
START_DATE = '2020-01-01'
END_DATE = '2024-01-01'

# Load market data
print("Loading market data...")
data = {}

for symbol in SYMBOLS:
    try:
        ticker = yf.Ticker(symbol)
        df = ticker.history(start=START_DATE, end=END_DATE)
        data[symbol] = df
        print(f"Loaded {symbol}: {len(df)} records")
    except Exception as e:
        print(f"Error loading {symbol}: {e}")

# Combine into a single DataFrame
price_data = pd.DataFrame()
for symbol, df in data.items():
    price_data[symbol] = df['Close']

# Display basic information
print(f"\nData shape: {price_data.shape}")
print(f"Date range: {price_data.index.min()} to {price_data.index.max()}")
price_data.head()

## 3. Momentum Indicators Calculation

Calculate various momentum indicators and signals.

In [ ]:
def calculate_momentum_indicators(prices, windows=[5, 10, 20, 50]):
    """
    Calculate various momentum indicators
    
    Parameters:
    prices (pd.DataFrame): Price data
    windows (list): List of lookback windows
    
    Returns:
    dict: Dictionary of momentum indicators
    """
    indicators = {}
    
    # Calculate returns
    returns = prices.pct_change()
    indicators['returns'] = returns
    
    # Time series momentum (own past returns)
    for window in windows:
        indicators[f'momentum_{window}d'] = prices.pct_change(window)
        indicators[f'sma_{window}d'] = prices.rolling(window).mean()
        indicators[f'volatility_{window}d'] = returns.rolling(window).std()
    
    # RSI (Relative Strength Index)
    for col in prices.columns:
        indicators[f'rsi_{col}'] = ta.momentum.RSIIndicator(prices[col]).rsi()
    
    # MACD
    for col in prices.columns:
        macd = ta.trend.MACD(prices[col])
        indicators[f'macd_{col}'] = macd.macd()
        indicators[f'macd_signal_{col}'] = macd.macd_signal()
        indicators[f'macd_histogram_{col}'] = macd.macd_diff()
    
    # Cross-sectional momentum (relative performance)
    for window in windows:
        momentum = prices.pct_change(window)
        # Rank-based momentum (percentile ranks)
        indicators[f'rank_momentum_{window}d'] = momentum.rank(axis=1, pct=True)
        # Z-score momentum
        indicators[f'zscore_momentum_{window}d'] = momentum.sub(momentum.mean(axis=1), axis=0).div(momentum.std(axis=1), axis=0)
    
    return indicators

# Calculate momentum indicators
momentum_indicators = calculate_momentum_indicators(price_data)
print(f"Calculated {len(momentum_indicators)} momentum indicators")

# Display some examples
print("\nMomentum indicators available:")
for key in list(momentum_indicators.keys())[:10]:
    print(f"- {key}")